In [10]:
import random, sys
import pygame
import time

# Constants -- cell marks
BOTTOMWALL = 0
RIGHTWALL  = 1
VISITED    = 2

NORTH = 0
SOUTH = 1
WEST  = 2
EAST  = 3

BLACK   =(0,0,0)
WHITE   =(255,255,255)
PURPLE  =(100,10,100)
RED    =(255,150,150)
YELLOW = (255 ,255 ,0)

WIDTH = 800
HEIGHT = 800

screen = pygame.display.set_mode((WIDTH, HEIGHT))
screen.fill(WHITE)

space=20
length=10
width=10



class Maze:
    

    def __init__( self, n_rows, n_cols ):
    
        self.n_rows = n_rows
        self.n_cols = n_cols
        
        self.root=[[[-1 for i in range(2)] for j in range(n_rows)] for k in range (n_cols)];
        self.visits=[[0 for i in range(n_rows)] for j in range(n_cols)];

        self.maze = [None]*n_rows
        print(len(self.maze))

        sizex=200+n_cols*space
        sizey=200+n_rows*space
        size=(sizex,sizey)
        #screen=pygame.display.set_mode(size)

        pygame.display.set_caption("MAZE!")
        clock = pygame.time.Clock()
        #screen.fill(WHITE)

        pygame.draw.rect(screen,BLACK,(99,99,sizex-198,sizey-198),2)

        pygame.display.flip()

        # Set up the hedge array - initially all walls intact
        for i in range(n_rows):
          self.maze[i] = [None]*n_cols
          for j in range(n_cols):
            self.maze[i][j] = [True,True,False] # BOTTOMWALL,RIGHTWALL,VISITED

        # Choose a random starting point
        currCol = random.randrange(n_cols)
        currRow = random.randrange(n_rows)

        # The searh can be quite deep
        if n_rows*n_cols > sys.getrecursionlimit():
          sys.setrecursionlimit( n_rows*n_cols+5 )

        # Recursively Remove Walls - Depth First Algorithm
        self._make_path( currRow, currCol )


  #*****************************************
    def _make_path( self, R, C, D=None ):
        

        maze = self.maze # speed up a bit

        # Knock out wall between this and previous cell
        maze[R][C][VISITED] = True;

        if   D==NORTH: maze[R]  [C]  [BOTTOMWALL] = False
        elif D==SOUTH: maze[R-1][C]  [BOTTOMWALL] = False
        elif D==WEST:  maze[R]  [C]  [RIGHTWALL]  = False
        elif D==EAST:  maze[R]  [C-1][RIGHTWALL]  = False

        # Build legal directions array
        directions = []
        if R>0            : directions.append(NORTH)
        if R<self.n_rows-1: directions.append(SOUTH)
        if C>0            : directions.append(WEST)
        if C<self.n_cols-1: directions.append(EAST)

        # Shuffle the directions array randomly
        dir_len = len(directions)
        for i in range(dir_len):
          j = random.randrange(dir_len)
          directions[i],directions[j] = directions[j],directions[i]

        # Call the function recursively
        for dir in directions:
          if dir==NORTH:
            if not maze[R-1][C][VISITED]:
              self._make_path( R-1,C,NORTH )
          elif dir==SOUTH:
            if not maze[R+1][C][VISITED]:
              self._make_path( R+1,C,SOUTH )
          elif dir==EAST:
            if not maze[R][C+1][VISITED]:
              self._make_path( R,C+1,EAST )
          elif dir==WEST:
            if not maze[R][C-1][VISITED]:
              self._make_path( R,C-1,WEST )
      #else: raise 'bug:you should never reach here'


  #*****************************************

    def __str__(self):
        """Return maze table in ASCII"""
        result = '' + self.n_cols*'__'
        result += '\n'
        for i in range(self.n_rows):
          result += '|'

          for j in range(self.n_cols):
            if i==self.n_rows-1 or self.maze[i][j][BOTTOMWALL]:
              result += '_'
            else:
              result += ' '
            if j==self.n_cols-1 or self.maze[i][j][RIGHTWALL]:
              result += '|'
            else:
              result += '_'
          result += '\n'

        return result
    
    
    def graphical(self):
        pygame.draw.line(screen,BLACK,[100,100],[100+space*self.n_cols,100])
        for i in range(self.n_rows):
            pygame.draw.line(screen,BLACK,[100,100+space*(i)],[100,100+space*(i+1)])
            for j in range(self.n_cols):
                if i==self.n_rows+1 or self.maze[i][j][BOTTOMWALL]:
                    pygame.draw.line(screen,BLACK,[101+space*(j),100+space*(i+1)],[101+space*(j+1),100+space*(i+1)])
                
                if j==self.n_cols+1 or self.maze[i][j][RIGHTWALL]:
                    pygame.draw.line(screen,BLACK,[100+space*(j+1),100+space*i],[100+space*(j+1),100+space*(i+1)])
                
        
               
    def visit_block(self,i,j):
        if not self.maze[i][j][BOTTOMWALL]:
            if self.visits[i+1][j]==0:
                self.visits[i+1][j]=1
                self.root[i+1][j]=[i,j]
                Maze.visit_block(self,i+1,j)
        if not self.maze[i][j][RIGHTWALL]:
            if self.visits[i][j+1]==0:
                self.visits[i][j+1]=1
                self.root[i][j+1]=[i,j]
                Maze.visit_block(self,i,j+1)
        if not self.maze[i][j-1][RIGHTWALL]:
            if self.visits[i][j-1]==0:
                self.visits[i][j-1]=1
                self.root[i][j-1]=[i,j]
                Maze.visit_block(self,i,j-1)
        if not self.maze[i-1][j][BOTTOMWALL]:
            if self.visits[i-1][j]==0:
                self.visits[i-1][j]=1
                self.root[i-1][j]=[i,j]
                Maze.visit_block(self,i-1,j)
    
    def find(self):
        for i in range (self.n_rows):
            for j in range (self.n_cols):
                if self.visits[i][j]==0:
                    self.visits[i][j]=1
                    Maze.visit_block(self,i,j)
 
                
    def draw_solution(self):
        [i,j]=[self.n_rows-1,self.n_cols-1]
        pygame.draw.rect(screen,PURPLE,[101+(i*space),101+(j*space),space,space],0)
        [i,j]=self.root[i][j]
        pygame.display.flip()
        while not [i,j]==[0,0]:
            pygame.draw.rect(screen,RED,[101+(j*space),101+(i*space),space,space],0)
            [i,j]=self.root[i][j]
            pygame.display.flip()
        pygame.draw.rect(screen,YELLOW,[101+(i*space),101+(j*space),space,space],0)
        pygame.display.flip()
            

   

#*****************************************
if __name__ == "__main__":

  syntax = ( "Syntax: %s [[-]n_rows [n_cols]]\n\n"
             "If n_cols is missing, it will be the same as n_rows\n"
             "If n_rows is negative, html representation will be generated\n\n"
             "Examples:\n%s 50 39 -- text maze 50 rows by 39 columns\n"
             "%s -40   -- html code of 40 x 40 cell maze"
           )


  # parse arguments if any

  import os.path
  import sys

  argc = len(sys.argv)
  name = os.path.basename( sys.argv[0] )

  if argc not in (2,3):
    print >>sys.stderr, syntax % (name,name,name)
    sys.exit(1)
  
  elif argc == 2:
    n_rows = n_cols = int(sys.argv[1])
  elif argc == 3:
    n_rows = int(len(sys.argv[1]))
    n_cols = int(len(sys.argv[2]))

  # create and print maze

  try:
    if n_rows > 0:
        maze=Maze(10,10)
        
        print (maze)
        
        
        maze.find()
        maze.draw_solution()
        maze.graphical()
        pygame.display.flip()
    else:
        maze = Maze( abs(n_rows), abs(n_cols) )
        
        
        maze.find()
        maze.draw_solution()
        maze.graphical()
        pygame.display.flip()
        print (maze.as_html_table())
      
  except MemoryError:
    print ("Sorry, n_rows, n_cols were too big")

10
____________________
| _ | _____ | _ ____|
| |_|____ |___|____ |
| _____ |__ |____ | |
| |__ |_____|__ |___|
| | ___ | _________ |
| | |_____| _ | ____|
| |__ | __| |___| _ |
| | __| | __| _ | | |
|___| _ |__ |_|___| |
|_____|_____________|

